In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms, models
import cv2


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os

In [0]:
os.listdir('/content/drive/My Drive/Accident')

['Data.zip', 'Real_test', 'Untitled0.ipynb']

In [0]:
!unzip '/content/drive/My Drive/Accident/Data.zip'

In [0]:
os.listdir('Data')

['test', '.DS_Store', 'train']

In [0]:
dataDirectory = 'Data'

In [0]:
trainTransform = transforms.Compose([transforms.RandomRotation(30),
                                    transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

testTransform = transforms.Compose([transforms.Resize(255),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

In [0]:
#Data
train_data = datasets.ImageFolder(dataDirectory+'/train',transform = trainTransform)
test_data = datasets.ImageFolder(dataDirectory+'/test',transform = testTransform)

#DataLoaders
trainloader = torch.utils.data.DataLoader(train_data ,batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)


In [0]:
model = models.densenet121(pretrained=True)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/checkpoints/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:00<00:00, 77603936.10it/s]


In [0]:
torch.cuda.is_available()

True

In [0]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0005)

model.to(device);


In [0]:
epochs = 2
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()


Epoch 1/2.. Train loss: 0.666.. Test loss: 0.648.. Test accuracy: 0.642
Epoch 1/2.. Train loss: 0.629.. Test loss: 0.623.. Test accuracy: 0.704
Epoch 1/2.. Train loss: 0.575.. Test loss: 0.612.. Test accuracy: 0.694
Epoch 1/2.. Train loss: 0.573.. Test loss: 0.607.. Test accuracy: 0.679
Epoch 1/2.. Train loss: 0.555.. Test loss: 0.613.. Test accuracy: 0.655
Epoch 1/2.. Train loss: 0.488.. Test loss: 0.608.. Test accuracy: 0.624
Epoch 1/2.. Train loss: 0.498.. Test loss: 0.614.. Test accuracy: 0.640
Epoch 1/2.. Train loss: 0.511.. Test loss: 0.630.. Test accuracy: 0.620
Epoch 1/2.. Train loss: 0.452.. Test loss: 0.616.. Test accuracy: 0.678
Epoch 1/2.. Train loss: 0.518.. Test loss: 0.658.. Test accuracy: 0.623
Epoch 1/2.. Train loss: 0.479.. Test loss: 0.641.. Test accuracy: 0.621
Epoch 1/2.. Train loss: 0.445.. Test loss: 0.584.. Test accuracy: 0.682
Epoch 1/2.. Train loss: 0.533.. Test loss: 0.756.. Test accuracy: 0.605
Epoch 2/2.. Train loss: 0.549.. Test loss: 0.573.. Test accuracy

In [0]:
os.listdir('/content/drive/My Drive/Accident/Real_test')

['a.jpg']

In [0]:
inTransform = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Resize(255),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

In [0]:
inference = cv2.imread('/content/drive/My Drive/Accident/Real_test/a.jpg')

In [0]:
inf = inTransform(inference)

In [0]:
inf = inf.unsqueeze(0)

In [0]:
device

device(type='cuda')

In [0]:
inf = inf.to(device)

In [0]:
logps = model.forward(inf)
ps = torch.exp(logps)
top_p, top_class = ps.topk(1, dim=1)

In [0]:
top_class

tensor([[1]], device='cuda:0')

In [0]:
path = '/content/drive/My Drive/Accident/Model' 

In [0]:
torch.save(model.state_dict() , path+'/checkpoint.pth')


In [0]:
model.state_dict()